In [ ]:
"""
UTC and Local

datetime.now(tz) 函数用于获取指定时区的当前时间
    tz 时区信息，默认为无时区
    
不指定 tz 参数时，返回一个 "local" 时间，即本地时间，不包含时区信息
"""

import datetime as dt
from dateutil import tz

t_loc = dt.datetime.now()  # 获取当前本地时间
t_utc = dt.datetime.now(tz=tz.tzutc())  # 获取当前 UTC 时间

print('* local time is {}'.format(t_loc))
print('* UTC time is {}'.format(t_utc))

In [ ]:
"""
强行替换时区

datetime.replace(tz) 可以在不改变时间值的前提下，更换一个新时区
"""

import datetime as dt
from dateutil import tz

t_loc = dt.datetime.now().replace(tzinfo=tz.tzutc())  # 将本地时间值不变，时区换为 UTC
t_utc = dt.datetime.now(tz=tz.tzutc())

print('* local time is {}'.format(t_loc))
print('* UTC time is {}'.format(t_utc))
print('* time diff is {}'.format(round(abs(t_loc.timestamp() - t_utc.timestamp()) / 3600, 4)))

t_loc = dt.datetime.now()
t_utc = dt.datetime.now(tz=tz.tzutc()).replace(tzinfo=None)  # 获取当前 UTC 时间并擦除时区信息

print('* local time is {}'.format(t_loc))
print('* UTC time is {}'.format(t_utc))
print('* time diff is {}'.format(round(abs(t_loc.timestamp() - t_utc.timestamp()) / 3600, 4)))

In [ ]:
"""
创建时区信息

tz.tzoffset(name, offset) 函数用于创建一个时区对象
    name 该时区的名称
    offset 该时区相对于 0 时区的时差，单位 秒
"""

import datetime as dt
from dateutil import tz

def time_diff(t1, t2):
    t1 = t1.replace(tzinfo=None)
    t2 = t2.replace(tzinfo=None)
    return round(abs(t1.timestamp() - t2.timestamp()) / 3600, 4)


tzinfo = tz.tzoffset('CST', 7 * 3600)  # 设置名为 CTS 的时区，和 0 时区差 7 小时
t_cst = dt.datetime(2015, 10, 1, 20, 22, 22, tzinfo=tzinfo)
t_utc = t_cst.astimezone(tz=tz.tzutc())
print('* cts time is {}'.format(t_cst))
print('* utc time is {}'.format(t_utc))
print('* time diff is {}'.format(time_diff(t_cst, t_utc)))

tzinfo = tz.tzoffset('PST', -7 * 3600)  # 设置名为 PTS 的时区，和 0 时区差 -7 小时
t_pts = t_cst.astimezone(tz=tzinfo)
print('* cts time is {}'.format(t_pts))
print('* time diff is {}'.format(time_diff(t_pts, t_utc)))

In [ ]:
"""
通过字符串定义时区信息

POSIX1003.1 timezone string format:

    TZ  Timezone information. TZ has the form:

        stdoffset[dst[offset],[start[/time],end[/time]]]

    std and dst
        Three or more bytes that are the designation for the standard(std) and daylight savings time
        (dst) timezones. Only std is required. If dst is missing, then daylight savings time does not
        apply in this locale. Upper- and lower-case letters are allowed. Any characters except a
        leading colon (:), digits, a comma (,), a minus (-) or a plus (+) are allowed.

    offset
        Indicates the value one must add to the local time to arrive at Coordinated Universal Time.
        The offset has the form:

            hh[:mm[:ss]]

        The minutes (mm) and seconds (ss) are optional. The hour (hh) is required and may be a single
        digit. The offset following std is required. If no offset follows dst , daylight savings time
        is assumed to be one hour ahead of standard time. One or more digits may be used; the value
        is always interpreted as a decimal number. The hour must be between 0 and 24, and the minutes
        (and seconds) if present between 0 and 59. Out of range values may cause unpredictable
        behavior. If preceded by a "-" the timezone is east of the Prime Meridian; otherwise it is west
        (which may be indicated by an optional preceding "+" sign).

    start/time, end/time
        Indicate when to change to and back from daylight savings time, where start/time describes
        when the change from standard time to daylight savings time occurs, and end/time describes when
        the change back happens. Each time field describes when, in current local time, the change is made.

        The formats of start and end are one of the following:

        Jn      The Julian day n (1 < n < 365). Leap days are not counted. That is, in all years,
                February 28 is day 59 and March 1 is day 60. It is impossible to refer to the
                occasional February 29.

        n       The zero-based Julian day (0 < n < 365). Leap days are counted, and it is possible
                to refer to February 29.

        Mm.n.d  The d**th day, (0 < d < 6) of week n of month m of the year (1 < n < 5, 1 < m < 12),
        where week 5 means "the last d-day in month m" which may occur in either the fourth or the fifth
        week). Week 1 is the first week in which the d**th day occurs. Day zero is Sunday.

        Implementation specific defaults are used for start and end if these optional fields are not given.

        The time has the same format as offset except that no leading sign ("-" or "+" is allowed. The
        default, if time is not given is 02:00:00.

    example: [continent/city] TZ=posix-tz-string
    1. without daylight saving example:
        [Pacific/Honolulu] TZ=HST10
        Where HST is the designation for the time zone (in this case Hawaii Standard Time) and
        10 is the offset in hours. The offset indicates the value one must add to the local
        time to arrive at Coordinated Universal Time (UTC, aka GMT), and so it is positive for
        west of the meridian, e.g. America, and negative for east, e.g. China.

        [Asia/Beijing] TZ=CST-8
        Minutes and seconds are optional, so CST-8 and CST-08:00:00 mean the same thing. Note
        that the sign convention (+/-) used in a Posix TZ string is the opposite to that used in
        Internet time offsets (RFC 3339) and in Arthur David Olson's TZ data files.

    2. with daylight saving time example:
        [America/New_York] TZ=EST5EDT,M3.2.0/2,M11.1.0
        EST = designation for standard time when daylight saving is not in force
        5 = offset in hours = 5 hours west of Greenwich meridian (i.e. behind UTC)
        EDT = designation when daylight saving is in force (if omitted there is no daylight saving)
        , = no offset number between code and comma, so default to one hour ahead for daylight saving
        M3.2.0 = when daylight saving starts = the 0th day (Sunday) in the second week of month 3 (March)
        /2, = the local time when the switch occurs = 2 a.m. in this case
        M11.1.0 = when daylight saving ends = the 0th day (Sunday) in the first week of month 11 (November).
                  No time is given here so the switch occurs at 02:00 local time.
                  So daylight saving starts on the second sunday in March and finishes on the first Sunday
                  in November. The switch occurs at 02:00 local time in both cases. This is the default
                  switch time, so the /2 isn't strictly needed.

        [Europe/Paris] TZ=CET-1CEST,M3.5.0/2,M10.5.0/3
        CET = designation for standard time when daylight saving is not in force
        -1 = offset in hours = negative so 1 hour east of Greenwich meridian
        CEST = designation when daylight saving is in force ("Central European Summer Time")
        , = no offset number between code and comma, so default to one hour ahead for daylight saving
        M3.5.0 = when daylight saving starts = the last Sunday in March (the "5th" week means the last
                 in the month)
        /2, = the local time when the switch occurs = 2 a.m. in this case
        M10.5.0 = when daylight saving ends = the last Sunday in October.
        /3, = the local time when the switch occurs = 3 a.m. in this case
              The European Union time zones are arranged so the switch takes place at the same time in
              all zones.

        [America/St_Johns] TZ=NST03:30NDT,M3.2.0/0:01,M11.1.0/0:01
        Newfoundland's standard time is three and a half hours behind UTC with daylight saving
        begin on the second Sunday in March and ending on the first Sunday in November, but
        the switch occurs at one minute past midnight local time.
        As an extreme example, consider the mythical city of Foobar in Atlantis.

        [Atlantis/Foobar] TZ=AST2:45ADT0:45,M4.1.6/1:45,M10.5.6/2:45
        Atlantis Standard Time (AST) is 2 hours 45 minutes behind UTC and for daylight saving (ADT)
        they put their clocks forward two hours (i.e. to be 45 minutes behind UTC). Daylight saving
        starts on the first Saturday in April with the switch happening at 01:45 in the morning, and
        daylight saving ends on the last Saturday in October with the switch at 02:45 local time.
"""

import datetime as dt
from dateutil import tz

# 定义一个时区，正常时间为东八区时间，夏令时为东 9 区时间，夏令时开始时间为 3 月第一个星期天到 5 月最后一个星期天
# M3.1.0 即 3 月第一周第 0 天（周日），M5.5.0即5月第5周（最后一周）第 0 天（周日）
tzinfo = tz.tzstr('RPC-8CRPC-9,M3.1.0/02:00,M5.5.0/10:00')

t_rpc = dt.datetime(2015, 10, 1, 20, 22, 22, tzinfo=tzinfo)  # 正常时间区间
print('* zone of time "{}" is "{}"'.format(t_rpc, tzinfo.tzname(t_rpc)))

t_crpc = dt.datetime(2015, 4, 1, 10, 0, 0, tzinfo=tzinfo)  # 夏令时时间区间
print('* zone of time "{}" is "{}"'.format(t_crpc, tzinfo.tzname(t_crpc)))